## 06 Transfert Learning with PyTorch

What is Transfer Learning?

Transfer learning is a powerful technique in machine learning where knowledge gained from solving one problem is applied to a different but related problem.  Instead of starting the training process from scratch, you leverage a pre-trained model that has already learned useful features from a large dataset. This pre-trained model serves as a starting point, and you fine-tune it on your specific task, which typically requires less data and computational resources than training a model from scratch.

Here's a breakdown of the key concepts:

**1. Pre-trained Model:** This is a model that has been trained on a massive dataset, usually for a general task like image classification (e.g., ImageNet).  These models have learned a rich set of features that can be useful for other related tasks.

**2. Feature Extraction:**  Pre-trained models are excellent feature extractors.  The early layers of the model learn general features (like edges, textures), while later layers learn more specific features (like shapes, objects).  You can use a pre-trained model to extract these features from your data without retraining the entire model.

**3. Fine-tuning:**  This involves taking a pre-trained model and adapting it to your specific task. You typically replace the final layer(s) of the pre-trained model with layers suited for your task (e.g., a new classification layer with the correct number of classes). Then, you train the modified model on your dataset.  You can choose to freeze the weights of the early layers (to preserve the general features) and only train the later layers, or you can train the entire model with a lower learning rate.

**4. When to Use Transfer Learning:**

* **Limited Data:**  Transfer learning is especially beneficial when you have a small dataset for your target task.  Training a complex model from scratch on limited data often leads to overfitting.
* **Computational Constraints:** Fine-tuning a pre-trained model is much faster and requires fewer resources than training from scratch.
* **Improved Performance:**  Transfer learning can often lead to significantly better performance, especially when the source and target tasks are closely related.

**Example:**

Imagine you want to build an image classifier to identify different types of flowers. You have a small dataset of flower images. Instead of training a CNN from scratch, you can use a pre-trained model like ResNet, which has been trained on ImageNet (a massive dataset of general images). You remove ResNet's final classification layer (which classifies ImageNet categories) and replace it with a new classification layer with the number of flower types you want to classify. Then, you fine-tune the modified ResNet on your flower image dataset. The pre-trained features learned by ResNet on ImageNet will help the model learn to classify flowers more efficiently and accurately.

**Key Benefits:**

* Faster training
* Reduced data requirements
* Improved performance (often)
* Easier prototyping

Transfer learning is a widely used and highly effective technique in deep learning, allowing you to leverage the power of large pre-trained models to solve a wide range of problems.


In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.4.1+cu121
0.19.1+cu121


In [4]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision.transforms import v2 as transforms

# Try to get torchinfo, install it if it doen't work
try:
    from torchinfo import summary
except:
    print("torchinfo not found, installing it")
    %pip install torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, if it doesn't work, clone the repo
try:
    from going_modular import data_setup, engine
except:
    print("going_modular not found, cloning the repo")


In [6]:
# Setup device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## 1. Get data

we need pizza, steak, sushi data to build our tranfert model

In [19]:
import os
import zipfile

from pathlib import Path

import requests

# Setup data path
data_dir = Path("data/")
image_dir = data_dir / "images"
zip_path = data_dir / "pizza_steak_sushi.zip"


# Download the data
url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
if not data_dir.exists():
    print("Downloading the data")
    # Ensure the data directory exists
    data_dir.mkdir(parents=True, exist_ok=True)
    response = requests.get(url)
    zip_path.write_bytes(response.content)
else:
    print("Data already downloaded")

# Unzip the data
if zip_path.exists():
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        print("Unzipping the data")
        zip_ref.extractall(data_dir)
        os.remove(zip_path)
else:
    print("Data already unzipped")

# remove the zip file


Data already downloaded
Data already unzipped


In [42]:
# Setup the data
train_dir = data_dir / "train"
test_dir = data_dir / "test"

train_dir, test_dir

(PosixPath('data/train'), PosixPath('data/test'))

## 2. Create Datasets and Dataloaders

Now we've got some data, we want to turn it into a form our model can use.

To do this, we'll use the `torchvision.datasets.ImageFolder` class. This class assumes the images are in the following structure:
    
    ```
    root/train/class1/xxx.png
    root/train/class2/xxy.png
    root/train/class3/xxz.png

    root/test/class1/123.png
    root/test/class2/nsdf3.png
    root/test/class3/asd932_.png
    ```

### 2.1 Creating a transform for `torchvision.models` models

In [35]:
from going_modular import data_setup

from torchvision.transforms import v2 as transforms

In [40]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])

manual_transform = transforms.Compose([
                                        transforms.Resize((224, 224)), 
                                        transforms.ToDtype(torch.float32, scale=True),
                                        normalize,
                                        transforms.ToImage()])

In [43]:
train_data, test_data, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                   test_dir=test_dir,
                                                                   transform=manual_transform,
                                                                   batch_size=32)

In [44]:
train_data, test_data, class_names

(<torch.utils.data.dataloader.DataLoader at 0x79869466e6c0>,
 ['pizza', 'steak', 'sushi'])

### 2.2 Creating a transform for `torchvision.models`

As of `torchvision` 0.10.0, the `torchvision.models` module has been updated to accept a `transform_input` parameter. This parameter allows you to pass in a function to transform the input data before it's passed through the model.

This is particularly useful when you're using a model which was trained on a different dataset to the one you're using. For example, the `torchvision.models` models were trained on the ImageNet dataset, which has different normalization values to the ones we're using for our pizza_steak_sushi dataset.

